In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.api._v1.keras.layers import Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.api._v1.keras.models import Sequential

# import matplotlib.pyplot as plt




# Read the CSV and Perform Basic Data Cleaning

In [3]:
orig_df = pd.read_csv("Resources/03_new_df.csv")
df = orig_df
df.describe()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30
count,1.907000e+03,1907.000000,1.907000e+03,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,...,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000
mean,2.792583e+07,2913.979549,6.206534e+07,110.750393,0.359203,0.280545,0.093865,0.409544,0.149449,0.004719,...,0.040378,0.031463,0.029890,0.045621,0.027792,0.027268,0.028841,0.026744,0.023073,0.015732
std,2.729695e+07,702.092481,5.495635e+07,18.674676,0.479893,0.449384,0.291717,0.491879,0.356624,0.068554,...,0.196895,0.174611,0.170328,0.208717,0.164421,0.162906,0.167404,0.161375,0.150174,0.124468
min,1.000198e+07,629.000000,0.000000e+00,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.339860e+07,2503.000000,2.400000e+07,97.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.896668e+07,2950.000000,4.600000e+07,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.053719e+07,3389.500000,8.000000e+07,122.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.571150e+08,4662.000000,5.000000e+08,201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df.head()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30
0,357115007,4662,356000000,181.0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,257698183,4474,300000000,149.0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,247966675,4134,245000000,136.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,220009584,4232,200000000,152.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,208806270,4274,150000000,124.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Train Test Split
Use `Opening` for the y values

In [5]:
target = df["Opening"]
data = df.drop("Opening", axis=1)
feature_names = data.columns
data.head()

,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,...,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30
0,4662,356000000,181.0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,4474,300000000,149.0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,4134,245000000,136.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4232,200000000,152.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4274,150000000,124.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
X_train.shape

(1430, 16312)

In [8]:
y_train.shape

(1430,)

# Pre-processing

Scale the data using the MinMaxScaler

In [9]:
scalers = {}
X_minmax = MinMaxScaler().fit(X_train)
scalers['X'] = X_minmax
# y_train.values.reshape(-1, 1)
y_minmax = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
scalers['Y'] = y_minmax

In [10]:
X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)
y_train_scaled = y_minmax.transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_minmax.transform(y_test.values.reshape(-1, 1))

In [ ]:
# X_mapper = DataFrameMapper([(['Theaters', 'Budget', 'Runtime'], MinMaxScaler())], 
#                           default=None).fit(X_train)

In [ ]:
# X_train_scaled = X_mapper.transform(X_train)
# X_test_scaled = X_mapper.transform(X_test)

In [11]:
X_train_scaled.shape

(1430, 16312)

In [12]:
X_train_scaled

array([[0.56586366, 0.06      , 0.45142857, ..., 0.        , 0.        ,
        0.        ],
       [0.67657611, 0.32      , 0.55428571, ..., 0.        , 0.        ,
        0.        ],
       [0.61455664, 0.08      , 0.51428571, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.21655561, 0.05      , 0.56571429, ..., 0.        , 0.        ,
        0.        ],
       [0.63172732, 0.034     , 0.34857143, ..., 0.        , 0.        ,
        0.        ],
       [0.65171707, 0.064     , 0.34857143, ..., 0.        , 0.        ,
        0.        ]])

In [13]:
y_train.shape

(1430,)

# Try simple linear regression

In [ ]:
# # Import dependencies
# from sklearn.linear_model import LinearRegression

# # Create a linear model
# model = LinearRegression()

# # Fit (Train) our model to the data
# model.fit(X_train_scaled, y_train_scaled)

In [ ]:
# predictions = model.predict(X_test_scaled)

In [ ]:
# # Overall Score for the model
# model.score(X_train_scaled, y_train_scaled)

In [ ]:
# model.score(X_test_scaled, y_test_scaled)

In [ ]:
# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = model.score(X_test_scaled, y_test_scaled)

# print(f"MSE: {MSE}, R2: {r2}")

# LASSO model

In [ ]:
# # LASSO model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import Lasso

# lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

# predictions = lasso.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = lasso.score(X_test_scaled, y_test_scaled)
# ### END SOLUTION

# print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
# lasso.score(X_train_scaled, y_train_scaled)

In [ ]:
# lasso.score(X_test_scaled, y_test_scaled)

# Ridge model

In [ ]:
# # Ridge model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import Ridge

# ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

# predictions = ridge.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = ridge.score(X_test_scaled, y_test_scaled)

# print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
# ridge.score(X_train_scaled, y_train_scaled)

In [ ]:
# ridge.score(X_test_scaled, y_test_scaled)

# ElasticNet model

In [ ]:
# # ElasticNet model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import ElasticNet

# ### BEGIN SOLUTION
# elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)
# model.score(X_train_scaled, y_train_scaled)

# model.score(X_test_scaled, y_test_scaled)
# predictions = elasticnet.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = elasticnet.score(X_test_scaled, y_test_scaled)
# ### END SOLUTION

# print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
# elasticnet.score(X_train_scaled, y_train_scaled)

In [ ]:
# elasticnet.score(X_test_scaled, y_test_scaled)

# Try Neural Net: Define the architecture

In [14]:
nn_model = Sequential()

In [15]:
nn_model.add(Dense(units=1500, input_dim=int(X_test_scaled.shape[1]), activation='relu'))
nn_model.add(Dense(units=800, activation='relu'))
nn_model.add(Dense(units=300, activation='relu'))
nn_model.add(Dense(units=1, activation='linear'))
nn_model.summary()

W0727 20:38:09.458629 4543567296 deprecation.py:506] From /Users/mikedodd/anaconda3/envs/Project-3-Opening-Weekend/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              24469500  
_________________________________________________________________
dense_1 (Dense)              (None, 800)               1200800   
_________________________________________________________________
dense_2 (Dense)              (None, 300)               240300    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 25,910,901
Trainable params: 25,910,901
Non-trainable params: 0
_________________________________________________________________


In [16]:
nn_model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [17]:
nn_model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=100,
    shuffle=False,
    verbose=2
)

Epoch 1/100
1430/1430 - 8s - loss: 0.0113 - mean_squared_error: 0.0113 - mean_absolute_error: 0.0583
Epoch 2/100
1430/1430 - 7s - loss: 0.0039 - mean_squared_error: 0.0039 - mean_absolute_error: 0.0334
Epoch 3/100
1430/1430 - 7s - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0284
Epoch 4/100
1430/1430 - 7s - loss: 0.0030 - mean_squared_error: 0.0030 - mean_absolute_error: 0.0269
Epoch 5/100
1430/1430 - 7s - loss: 0.0016 - mean_squared_error: 0.0016 - mean_absolute_error: 0.0186
Epoch 6/100
1430/1430 - 8s - loss: 0.0016 - mean_squared_error: 0.0016 - mean_absolute_error: 0.0176
Epoch 7/100
1430/1430 - 8s - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0190
Epoch 8/100
1430/1430 - 9s - loss: 0.0023 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0200
Epoch 9/100
1430/1430 - 9s - loss: 0.0026 - mean_squared_error: 0.0026 - mean_absolute_error: 0.0263
Epoch 10/100
1430/1430 - 7s - loss: 0.0015 - mean_squared_error: 0.0015 - mean_absolute_err

In [18]:
nn_model.evaluate(
    X_test_scaled, y_test_scaled, verbose=2)

477/477 - 1s - loss: 0.0019 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0297


[0.0019235607608567057, 0.0019235607, 0.029700931]

In [19]:
nn_model.save("Resources/nn_model.h5")

In [20]:
pd.to_pickle(scalers, "Resources/scalers.pickle")
pd.to_pickle((X_train_scaled, y_train_scaled), "Resources/train_scaled.pickle")
pd.to_pickle((X_test_scaled, y_test_scaled), "Resources/test_scaled.pickle")

In [21]:
prev_col = None
count = 0
for c in data.columns:
    cg = str(c).split("_")[0]
    if cg != prev_col:
        if prev_col:
            print(f"{prev_col} => {count}")
        prev_col = cg
        count = 0
    count += 1
else:
    print(f"{prev_col} => {count}")

Theaters => 1
Budget => 1
Runtime => 1
Genre => 18
Director => 2738
Producer => 10728
Writer => 2732
Studio => 45
Rating => 5
Month => 12
Day => 31
